In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
!pip install --upgrade tensorflow-hub

Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.6/dist-packages (0.10.0)


In [21]:
!pip install spektral

     |████████████████████████████████| 112kB 9.7MB/s 


In [2]:
import numpy as np
import tensorflow as tf
import argparse
import scipy.sparse as sp
import pandas as pd 
import random
import json
import os

# from feature_matrix import FeatureMatrix

import pandas as pd 
import tensorflow_hub as hub
from bert import run_classifier
from bert import tokenization


In [3]:
##Code referenced from: 
##Website Title: BERT in Keras with Tensorflow hub
##URL : https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b

def create_tokenizer_from_hub_module():
  with tf.Graph().as_default():
    bert_module = hub.Module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.compat.v1.Session() as sess:
        vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                              tokenization_info["do_lower_case"]])

  return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [4]:
    def getFeatures( dataset="BuzzFeed"):
        feature_df =  FM.get_feature_matrix(dataset)
        label = feature_df['label'].tolist()
        label_comp = [0 if each else 1 for each in label]
        global label_zip
        label_zip = list(zip(label_comp, label))
        feature_df.drop(['label'], axis=1)
        feature_np = feature_df.values

        return sp.csr_matrix(feature_np, dtype=float).tolil()

In [5]:
def getYs():

  random.seed(1)
  yTrain =  label_zip[:]
  yVal =  label_zip[:]
  yTest =  label_zip[:]
  train_mask = [False] * len(yTrain)
  val_mask = [False] * len(yTrain)
  test_mask = [False] * len(yTrain)
  n = len(yTrain)

  set_of_records_range = set(range(n))

  train_range = set(random.sample(set_of_records_range, k=int(n * 0.6)))
  set_of_records_range = set_of_records_range - train_range

  val_range = set(random.sample(set_of_records_range, k=int(n * 0.2)))
  set_of_records_range = set_of_records_range - train_range

  test_range = set(random.sample(set_of_records_range, k=int(n * 0.2)))

  for i in train_range:
      yVal[i] = (0,0)
      yTest[i] = (0,0)
      train_mask[i] = True
  for i in val_range:
      yTrain[i] = (0,0)
      yTest[i] = (0,0)
      val_mask[i] = True
  for i in test_range:
      yVal[i] = (0,0)
      yTrain[i] = (0,0)
      test_mask[i] = True

  return yTrain, yVal, yTest, train_mask, val_mask, test_mask


In [6]:
def sparse_to_tuple(sparse_mx):
  """Convert sparse matrix to tuple representation."""
  def to_tuple(mx):
    if not sp.isspmatrix_coo(mx):
        mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    values = mx.data
    shape = mx.shape
    return coords, values, shape

  if isinstance(sparse_mx, list):
    for i in range(len(sparse_mx)):
        sparse_mx[i] = to_tuple(sparse_mx[i])
  else:
    sparse_mx = to_tuple(sparse_mx)

  return sparse_mx

In [7]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1, dtype=float).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense(), sparse_to_tuple(features)

In [8]:
base_path = "/content/drive/MyDrive/Colab_Notebooks/DL_final/"
folder = "BuzzFeed"
data_list = []
for subfolder in ["FakeNewsContent", "RealNewsContent"]:
  print("Getting data from subfolder: ", subfolder)
  file_array = [f for f in os.listdir(base_path + folder + "/" + subfolder) if f.endswith('.json')]
  file_array.sort() # file is sorted list
  file_array = [os.path.join(base_path + folder + "/" + subfolder, name) for name in file_array]

  for file in file_array:
    # print("file: ", file)
    # print("path: ",base_path + folder + "/" + subfolder)
    with open(file, 'r') as json_file:
      data = json.load(json_file)
      if file.split("/")[-2] == "FakeNewsContent":
          data_list.append([data['text'], 1])
      else:
          data_list.append([data['text'], 0])

print("Creating data frame")
data_frame = pd.DataFrame(data_list, columns=["text", "label"])
data_frame = data_frame.sample(frac=1)

input = data_frame.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                       text_a=x['text'], text_b=None, label=x['label']), axis=1)
print("extracting features")
tokenizer = create_tokenizer_from_hub_module()
features = run_classifier.convert_examples_to_features(input, [0, 1], 128, tokenizer)

train_features_list = []
for item in features:
  temp = item.input_ids
  temp.append(item.label_id)
  train_features_list.append(temp)
column_names = ["feature" + str(i) for i in range(128)]
column_names.append("label")
features_frame = pd.DataFrame(train_features_list, columns=column_names)  ##extracted features data frame
# /content/drive/MyDrive/Colab_Notebooks/DL_final/BuzzFeed/FakeNewsContent/BuzzFeed_Fake_3-Webpage.json

label = features_frame['label'].tolist()
label_comp = [0 if each else 1 for each in label]
global label_zip
label_zip = list(zip(label_comp, label))
features_frame.drop(['label'], axis=1)
feature_np = features_frame.values

Getting data from subfolder:  FakeNewsContent
Getting data from subfolder:  RealNewsContent
Creating data frame
extracting features
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 182


INFO:tensorflow:Writing example 0 of 182


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [9]:
features =  sp.csr_matrix(feature_np, dtype=float).tolil()
adj =  pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/DL_final/news_news_bf_adjacency_matrix.csv", header=None).values
y_train, y_val, y_test, train_mask, val_mask, test_mask =  getYs()

features=sp.csr_matrix(features)

features, sparse_matrix = preprocess_features(features)

In [10]:
y_train  = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)
train_mask  = np.asarray(train_mask)
val_mask  = np.asarray(val_mask)
test_mask  = np.asarray(test_mask)
label = np.asarray(label)

In [11]:
# np.isnan(y_train.data).any()
# np.isnan(y_val.data).any()
np.isnan(y_test.data).any()

False

In [12]:
train_mask.shape

(182,)

In [13]:
from __future__ import division
from __future__ import print_function
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import time
import tensorflow as tf 
from spektral.layers import GCNConv
# from  utils import *
# from models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.random.set_seed(seed)

epochs = 200
dropout = 0.2
weight_decay = 5e-4
early_stopping = 10
learning_rate = 1e-2
num_nodes = features.shape[0]
# num_nodes = adj.shape[0]
l2_reg = 5e-4 / 2
num_classes = 2
# 'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label = label_encoder.fit_transform(label)
label = tf.keras.utils.to_categorical(label)

In [14]:
features.shape[1]

129

In [34]:
adj_matrix = GCNConv.preprocess(adj).astype('f4')
x_inputs = Input(shape = features.shape[1])
a_inputs = Input((num_nodes,), sparse=True, dtype=tf.float32)
do_1 = Dropout(dropout)(x_inputs)
gc_1 = GCNConv(8,
               activation='relu',
               kernel_regularizer=l2(l2_reg),
               use_bias=False)([do_1, a_inputs])
# do_2 = Dropout(dropout)(gc_1)
# gc_2 = GCNConv(8,
#                activation='relu',
#                kernel_regularizer=l2(l2_reg),
#                use_bias=False)([do_2, a_inputs])

output =  Dense(units=num_classes, activation='softmax')(gc_1)
model = Model(inputs=[x_inputs, a_inputs], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()



Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 129)]        0                                            
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 129)          0           input_7[0][0]                    
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 182)]        0                                            
__________________________________________________________________________________________________
gcn_conv_6 (GCNConv)            (None, 8)            1032        dropout_6[0][0]                  
                                                                 input_8[0][0]         

In [35]:
tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./Tensorboard_GCN_cora',
)
callback_GCN = [tbCallBack_GCN]

In [36]:
print('adj shape', type(adj))

print('features shape', type(features))
print('y_train shape', type(y_train))

print('y_val shape', type(y_val))
print('y_test shape', type(y_test))
print('train_mask shape', type(train_mask))
print('val_mask shape', type(val_mask))
print('test_mask shape', type(test_mask)) 
print('label', type(label))


adj shape <class 'numpy.ndarray'>
features shape <class 'numpy.matrix'>
y_train shape <class 'numpy.ndarray'>
y_val shape <class 'numpy.ndarray'>
y_test shape <class 'numpy.ndarray'>
train_mask shape <class 'numpy.ndarray'>
val_mask shape <class 'numpy.ndarray'>
test_mask shape <class 'numpy.ndarray'>
label <class 'numpy.ndarray'>


In [37]:
print(np.isnan(y_train).any())
print(np.isnan(y_val).any())
print(np.isnan(y_test).any())
print(np.isnan(train_mask).any())
print(np.isnan(val_mask).any())
print(np.isnan(test_mask).any())
print(np.isnan(label).any())
print(np.isnan(features).any())
print(np.isnan(adj_matrix).any())

False
False
False
False
False
False
False
False
False


In [38]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)


print(label.shape)

(182, 2)
(182, 2)
(182, 2)
(182, 2)


In [39]:
# Train model
validation_data = ([features, adj_matrix], label, val_mask)
model.fit([features, adj_matrix],
          label,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=num_nodes,
          validation_data=validation_data,
          shuffle=False,
          )


# callbacks=[
#               EarlyStopping(patience=early_stopping,  restore_best_weights=True),
#               tbCallBack_GCN
#           ]
# history = model.fit(
#     features, 
#     y_train, 
#     sample_weight=tf.cast(train_mask, tf.float32), # This will be used in loss calculations
#     validation_data=(features, y_val, val_mask),
#     epochs=epochs, 
#     batch_size=num_nodes, # This is unusual in ML - since our adjacency matrix is the whole graph, we want to feed in the whole node_state array in each training step
#     verbose=1,
#     shuffle=False # Do not shuffle the order of our input data, since its order matches up to the adjacency matrix
# )


Epoch 1/200
1/1 [==============================] - 0s 144ms/step - loss: nan - acc: 0.4771 - val_loss: nan - val_acc: 0.5278
Epoch 2/200
1/1 [==============================] - 0s 27ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 3/200
1/1 [==============================] - 0s 20ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 4/200
1/1 [==============================] - 0s 18ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 5/200
1/1 [==============================] - 0s 18ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 6/200
1/1 [==============================] - 0s 17ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 7/200
1/1 [==============================] - 0s 18ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 8/200
1/1 [==============================] - 0s 16ms/step - loss: nan - acc: 0.5229 - val_loss: nan - val_acc: 0.5278
Epoch 9